In [1]:
#import gensim
from gensim import corpora, models, similarities, utils
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import pymysql
import re
import pyLDAvis.gensim
import pyLDAvis
import KAsql2 as ka

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# from KAsql import query_SQL
sql = 'SELECT id,business_id,stars,comment FROM review;' 
rows = ka.query_SQL(sql)#returns a tuple within a tuple: each row is a tuple, inside one big tuple.

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

USE ortho;


In [4]:
print(documents[0])# prints one review

Some background: I rolled my ankle and had a really nasty sprain on Sunday. I tried to R.I.C.E. but had this nagging feeling that I should get it checked out. I had gone with a friend's recommendation with a doc but she was on vacation for 2 weeks and her assistant couldn't offer an honest recommendation to another doctor. Sooo here's how Yelp comes into play. Dr Chen has great rates so why not. His friendly staff sets me up with an appointment same day after I explain my situation. I show up and this is definitely a busy office. For people who complain about waiting past their appointment time: Well... don't see a populargood doctor. There's a reason everyone is waiting. I use to work in doctor offices and they do try their best to stay on time but emergencies come up. Okay, moving on. There were some unknowns about my insurance plan but guess what: you pay for medical care. I care that my doctor is competent, has good bedside manners, and isn't recommending outrageous surgeries. Dr. 

In [5]:
def canonicalize(word):    
    bads = ['dr.','dr',"n't","'ve","'re",'wo',"'m"]
    goods = ['doctor','doctor','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [6]:
def unify_pronouns(word):
    """ map many onto one for pronouns - parts of speech"""
    many = ['he',"his","him","she",'her',"hers",
            'I','i','me','mine','my',"i'm","i've",
            'you','your','yours',"you're","you've",
            'it','its',"it's",
            'they','their','theirs',"they're","they've",
            'we','us','our','ours',"we're","we've"]
    few = ['male','male','male','female','female','female',
           'firstper','firstper','firstper','firstper','firstper','firstper','firstper',
           'secper','secper','secper','secper','secper',
           'neut3rd','neut3rd','neut3rd',
           'neut3rd','neut3rd','neut3rd','neut3rd','neut3rd',
          'coll3rd','coll3rd','coll3rd','coll3rd','coll3rd','coll3rd']
    try:
        idx = many.index(word)
        word=few[idx]
    except ValueError as e:
        word=word
    return word

In [7]:
def numwords():
    #add numbers to common words
    nums = np.arange(0,100)
    ss =''
    ss = ss.join([' '+str(n) for n in nums])
    return ss

In [12]:
def ids_to_names():
# Purpose of function is to exclude proper names of doctors - first and last names
# Function will accept list of yelp_ids extracted by mysql from business table
# Function will put together a list of names
# Function will exclude terms like family doctor and pediatrics

    dashed_names=[]#examples = ['tamalpais-pediatrics-novato','julie-doctor-kim']
    sql = 'SELECT DISTINCT(yelp_id) FROM business;' 
    yelp_ids = ka.query_SQL(sql)# extracts unique yelp_ids
    for i in range(0,len(yelp_ids)):
        dashed_names.append(yelp_ids[i][0])
    
    def split_append(yelp_id):
        yelp_id = yelp_id.split('-')
        #print(yelp_id)
        return yelp_id

#     business_type = ['pediatrics','family','medicine','doctor','institute','center',
#                      'practice','md','san','francisco','ucsf','one','medical','group',
#                     'sports','fitness','therapy','acupuncture','chiropractic','district']
    
    names = []
#     for name in dashed_names:
#         [names.append(l) for l in split_append(name) if l not in business_type]
    for name in dashed_names:
        [names.append(l) for l in split_append(name)]
    return names

In [13]:
def get_pat_for_tokenizer():
    from nltk.tokenize import regexp_tokenize
    """money and time patterns formatted to fit in nltk RegexpTokenizer object, debugged with s2 below"""
    #s2 = 'it took until 12:30 am to get an appointment and I spent $3.50'
    
    pat = '\w+|\$[\d\.]+|\S+'
    pat2 = '\d+\:[\d]+\s?(pm|am)|\S+'

    pat3 = '('+pat2+')' +'|'+ '('+pat+')'#must be one string argument... | is OR
    #tokenizer = RegexpTokenizer(pat3)#time
    return pat3

In [73]:
# CLEAN & TOKENIZE - stoplists, lowercase, regular expressions, time-expressions

# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer

#texts: A list of lists; text: list of tokenized words from 1 doc/review
#document = a string

#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')#money
#tokenizer = RegexpTokenizer('\d*'+'\:'+'\d*')#time
tokenizer = RegexpTokenizer(get_pat_for_tokenizer())#time & money
mystopwords = stopwords.words() + ids_to_names()
#print(mystopwords[-500:])


texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower().replace('.',' ')) if word not in mystopwords]
        for document in documents]#review in reviews 

USE ortho;


In [74]:
# texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower()) if word not in stopwords.words()]
#         for document in documents]#review in reviews 


print(texts[:2])

[['background:', 'rolled', 'ankle', 'really', 'nasty', 'sprain', 'sunday', 'tried', 'r', 'nagging', 'feeling', 'get', 'checked', 'gone', "friend's", 'recommendation', 'doc', 'vacation', 'weeks', 'assistant', "couldn't", 'offer', 'honest', 'recommendation', 'another', 'doctor', 'sooo', "here's", 'yelp', 'comes', 'play', 'great', 'rates', 'friendly', 'staff', 'sets', 'appointment', 'day', 'explain', 'situation', 'show', 'definitely', 'busy', 'office', 'people', 'complain', 'waiting', 'past', 'appointment', 'time:', 'well', "don't", 'see', 'populargood', 'doctor', "there's", 'reason', 'everyone', 'waiting', 'use', 'work', 'doctor', 'offices', 'try', 'best', 'stay', 'time', 'emergencies', 'okay,', 'moving', 'unknowns', 'insurance', 'plan', 'guess', 'what:', 'pay', 'medical', 'care', 'care', 'doctor', 'competent,', 'good', 'bedside', 'manners,', "isn't", 'recommending', 'outrageous', 'surgeries', 'warm,', 'funny,', 'made', 'sure', 'knew', 'getting', 'take', 'care', 'granted', 'insurance', '

In [75]:
# Context
#result = [nltk.pos_tag(text) for text in texts[:2]]
#print result

simtext = nltk.Text(word for word in texts[1]) 
print(simtext.similar('help')) 

#print(simtext, np.shape(simtext))
#print(simtext.similar('pain'))
#simtext.similar('nasty')



None


In [69]:
# Parts of Speech
result = [nltk.pos_tag(text) for text in texts[:2]]
#nltk.help.upenn_tagset('RB')
print result

[[('background:', 'NN'), ('rolled', 'VBD'), ('ankle', 'NN'), ('really', 'RB'), ('nasty', 'JJ'), ('sprain', 'NN'), ('sunday', 'NN'), ('tried', 'VBD'), ('r', 'NN'), ('nagging', 'NN'), ('feeling', 'VBG'), ('checked', 'VBN'), ('gone', 'NN'), ("friend's", '-NONE-'), ('recommendation', 'NN'), ('vacation', 'NN'), ('weeks', 'NNS'), ('assistant', 'VBP'), ('offer', 'NN'), ('honest', 'JJS'), ('recommendation', 'NN'), ('another', 'DT'), ('sooo', 'NN'), ('comes', 'VBZ'), ('play', 'NN'), ('great', 'JJ'), ('rates', 'NNS'), ('friendly', 'RB'), ('staff', 'NN'), ('sets', 'NNS'), ('appointment', 'NN'), ('day', 'NN'), ('explain', 'NN'), ('situation', 'NN'), ('show', 'NN'), ('definitely', 'RB'), ('busy', 'JJ'), ('office', 'NN'), ('people', 'NNS'), ('complain', 'VBP'), ('waiting', 'VBG'), ('past', 'JJ'), ('appointment', 'NN'), ('time:', 'NN'), ('well', 'RB'), ('populargood', 'NN'), ('reason', 'NN'), ('everyone', 'NN'), ('waiting', 'VBG'), ('use', 'NN'), ('work', 'NN'), ('offices', 'NNS'), ('best', 'JJS'), (

In [76]:
# BAG OF WORDS

useless = ['dr.','dr','doc','doctor','doctors',"doctor's","doctors'",'get','see','could','would','have','had','did','yelp','try',
                "there's",'ever',"here's",'here',"couldn't",'go','got','thing','things','made',"don't",'do','done','made','make',
                "it's",'went','\&','went','surgery','surgical','surgury','surgeon','surgeons','one',"he's","she's","he'd","she'd"
                "i'd","i'm",'it']#type list

dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
dictionary.filter_extremes(no_below=100, no_above=.70)
dictionary.filter_tokens(bad_ids=useless)

dictionary.save('/Users/kaschbacher/insightproject/yelp/git-yelp/orthopedists.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
#print(dictionary.token2id)# very long!

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(985 unique tokens: [u'cares', u'consider', u'saying', u'better,', u'lack']...)


In [77]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpus_mm = corpora.MmCorpus.serialize('orthopedists_bow.mm', corpus) # store to disk, for later use

# corpus looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# (word order index, frequency of occurence?)

In [78]:
# LDA  (takes awhile to run)
# The constructor estimates Latent Dirichlet Allocation model parameters based on a training corpus 
# id2word is a mapping from word ids (integers) to words (strings). 

# 1) The constructor - obtains LDA model parameters given training corpus
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=30, update_every=0, passes=10, iterations=100)


In [64]:
# SAVE THE LDA OBJECT
# --- note - i tried to do this with gensim, but it wasn't working, and ultimately cpickle did
#from gensim.utils import SaveLoad
#save(fname_or_handle, separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
#utils.SaveLoad.save(lda, pickle_protocol=2)

import cPickle as pickle
pickle_out = open('lda_ortho_30t_filter.p', 'wb')
pickle.dump(lda, pickle_out)
pickle_out.close()

In [65]:
# PRINT THE TOPICS (list of tuples)
lda.top_topics(corpus,num_words=6)

[([(0.034104928254396386, u'knee'),
   (0.02990391005406956, u'pain'),
   (0.015036742728605662, u'back'),
   (0.010701410672430906, u"he's"),
   (0.010146910715186216, u'never'),
   (0.010074120352515422, u"i'm")],
  -31.534049860965393),
 ([(0.01434923001873188, u'shoulder'),
   (0.014306231448542603, u'really'),
   (0.012390008175860039, u'even'),
   (0.011669690655508633, u'office'),
   (0.011360082784036369, u'never'),
   (0.011159212464489372, u'staff')],
  -31.696021844439819),
 ([(0.014606743023316696, u'staff'),
   (0.012206534563304509, u'right'),
   (0.011737760474094569, u'office'),
   (0.010293706745013075, u'one'),
   (0.0085341518053036625, u'good'),
   (0.0080444298333192041, u'recovery')],
  -32.448873068186614),
 ([(0.019001748665408599, u'pain'),
   (0.017713628999574026, u'good'),
   (0.015904670231165333, u'one'),
   (0.013850624793234317, u"i'm"),
   (0.0110633702070387, u"didn't"),
   (0.010559305434335297, u'experience')],
  -32.529014972840088),
 ([(0.011066230

In [43]:
# GAMMAS MATRIX: REVIEWS BY TOPICS, topic distributions over each review, given as proportions of all topics within a review
gammas,_ = lda.inference(corpus)# first element returned by lda.inference is gammas matrix: #shape (3616,10)
# Normalize:  sum over topics should equal one
norm_gammas = gammas/(np.sum(gammas,axis=1)[:,None])# Axis=1 = sum over columns for each row; Axis=0 sum over rows in each column
print(sum(norm_gammas[0][:]))#Yes, I checked that =1 - normalization worked
norm_gammas_df = pd.DataFrame(norm_gammas)
gammas_df = pd.DataFrame(gammas)

print(norm_gammas[:1][:10])

1.0
[[  3.00300300e-04   3.00300300e-04   3.00300300e-04   3.00300300e-04
    3.00300300e-04   3.00300300e-04   3.00300300e-04   3.00300300e-04
    3.00300300e-04   3.00300300e-04   9.91291291e-01   3.00300300e-04
    3.00300300e-04   3.00300300e-04   3.00300300e-04   3.00300300e-04
    3.00300300e-04   3.00300300e-04   3.00300300e-04   3.00300300e-04
    3.00300300e-04   3.00300300e-04   3.00300300e-04   3.00300300e-04
    3.00300300e-04   3.00300300e-04   3.00300300e-04   3.00300300e-04
    3.00300300e-04   3.00300300e-04]]


In [44]:
print(gammas[0][0:10])#row 0, all gammas


[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333]


In [45]:
# Add the R-ID (primary key) and B-ID (foreign key) to the NORM-GAMMAS DF
sql = 'SELECT id,business_id FROM review'+';'
rows = ka.query_SQL(sql)# returns shape #reviews x 2

# check that size are equivalent
if np.shape(norm_gammas_df)[0] !=np.shape(rows)[0]:
    print('Problem - rows in norm_gammas_df and sql query are not identical.')
else:
    #Convert sql return to array, and horizontally concatenate with gammas array, then convert to df
    ids_array = np.array(rows)
    
    # Norm-gammas
    ids_ngammas = np.hstack((ids_array, norm_gammas))#combine ids with norm gammas in numpy array
    ids_ngammas_df = pd.DataFrame(ids_ngammas)
    #---Rename columns (so topic # matches col) and confirm col-names, df.shape
    ids_ngammas_df.columns = [np.arange(-1,ids_ngammas_df.shape[1]-1)]
    ids_ngammas_df = ids_ngammas_df.rename(columns={-1: 'RID', 0: 'BID'})
    
    # Gammas
    ids_gammas = np.hstack((ids_array, gammas))#combine ids with norm gammas in numpy array
    ids_gammas_df = pd.DataFrame(ids_gammas)
    #---Rename columns (so topic # matches col) and confirm col-names, df.shape
    ids_gammas_df.columns = [np.arange(-1,ids_gammas_df.shape[1]-1)]
    ids_gammas_df = ids_gammas_df.rename(columns={-1: 'RID', 0: 'BID'})
    
    print(ids_ngammas_df.columns)
    print(ids_ngammas_df.shape)

USE ortho;
Index([u'RID', u'BID',      1,      2,      3,      4,      5,      6,      7,
            8,      9,     10,     11,     12,     13,     14,     15,     16,
           17,     18,     19,     20,     21,     22,     23,     24,     25,
           26,     27,     28,     29,     30],
      dtype='object')
(7320, 32)


In [46]:
# SAVE THE NORM_GAMMAS DATAFRAME OBJECT

import cPickle as pickle
# pickle_out = open('norm_gammas_df.p', 'wb')
# pickle.dump(norm_gammas_df, pickle_out)
pickle_out = open('ids_ngammas_df.p', 'wb')
pickle.dump(ids_ngammas_df, pickle_out)
pickle_out = open('ids_gammas_df.p', 'wb')
pickle.dump(ids_gammas_df, pickle_out)

pickle_out = open('norm_gammas.p', 'wb')
pickle.dump(norm_gammas, pickle_out)
pickle_out = open('gammas.p', 'wb')
pickle.dump(gammas, pickle_out)
pickle_out.close()

In [47]:
# CALCULATE AVG topic distribution (gamma vector) for each BID (averaging over RID)
# Should this be a new program, and just import ids_gammas_df?  Yes I think

bid_gmeans = ids_gammas_df.drop('RID', 1)
bid_gmeans = bid_gmeans.groupby('BID',axis=0,as_index='BID').mean()
#bid_gmeans.columns = [np.arange(1,bid_gmeans.shape[1]+1)]# not needed, since I renamed columns of ids_gammas_df instead

# Kept for reference syntax/comments
###bid_gamma_means = bid_gamma_means.drop(bid_gamma_means.index[0])# No, this actually just deletes a real row of data, but it is the right syntax
###bid_gamma_means.shape[0]#number of rows.  Matches: mysql query:  select count(distinct(business_id)) from review;

# Print df to inspect
#print(bid_gamma_means.loc[:,:])# works but prints everything
#bid_gamma_means.head(5)
bid_gmeans.tail(5)

#bid_gmeans = bid_gmeans.sort([np.arange(1,(bid_gmeans.shape[1])+1)], ascending=False)#doesn't work
bid_gmeans = bid_gmeans.sort([1,2,3,4,5],ascending=False)
bid_gmeans.head(5)
ids_ngammas_df.head(5)

# SAVE (ADDED to .GITIGNORE)
# pickle_out = open('bid_gmeans.p', 'wb')
# pickle.dump(bid_gmeans, pickle_out)
# pickle_out.close()

,RID,BID,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,13086,3,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,...,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300
1,13087,3,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,...,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606,0.000606
2,13088,3,0.993598,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221,...,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221,0.000221
3,13089,3,0.000303,0.000303,0.000303,0.000303,0.000303,0.000303,0.000303,0.000303,...,0.000303,0.000303,0.000303,0.000303,0.991212,0.000303,0.000303,0.000303,0.000303,0.000303
4,13090,3,0.962821,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,...,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282


In [48]:
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)#html object

In [51]:
#Save figure object. 
pickle.dump(vis_data, open('vis_data_30t.p','wb'))